<a href="https://colab.research.google.com/github/iamsoroush/DeepEEGAbstractor/blob/master/st_dfb_tests_4s_hmdd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title # Clone the repository and upgrade Keras {display-mode: "form"}

!git clone https://github.com/iamsoroush/DeepEEGAbstractor.git
!pip install --upgrade keras

Cloning into 'DeepEEGAbstractor'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 604 (delta 41), reused 45 (delta 25), pack-reused 541
Receiving objects: 100% (604/604), 28.53 MiB | 21.61 MiB/s, done.
Resolving deltas: 100% (388/388), done.
     |████████████████████████████████| 378kB 6.6MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [0]:
!rm -r DeepEEGAbstractor

In [0]:
#@title # Imports {display-mode: "form"}

import os
import pickle
import sys
sys.path.append('DeepEEGAbstractor')

import numpy as np

from src.helpers import CrossValidator
from src.models import DeepEEGAbstractor
from src.dataset import DataLoader, Splitter, FixedLenGenerator

from google.colab import drive
drive.mount('/content/gdrive')

tensorflow version:  1.15.0
tensorflow version:  1.15.0


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#@title # Set data path {display-mode: "form"}

#@markdown ---
#@markdown Type in the folder in your google drive that contains numpy _data_ folder:

parent_dir = 'soroush'#@param {type:"string"}
gdrive_path =  os.path.abspath(os.path.join('gdrive/My Drive', parent_dir))
data_dir = os.path.join(gdrive_path, 'data')
cv_results_dir = os.path.join(gdrive_path, 'cross_validation')
if not os.path.exists(cv_results_dir):
    os.mkdir(cv_results_dir)

print('Data directory: ', data_dir)
print('Cross validation results dir: ', cv_results_dir)

Data directory:  /content/gdrive/My Drive/soroush/data
Cross validation results dir:  /content/gdrive/My Drive/soroush/cross_validation


In [0]:
#@title ## Set Parameters

batch_size = 80
epochs = 80
k = 10
t = 10
instance_duration = 4 #@param {type:"slider", min:3, max:10, step:0.5}
instance_overlap = 1 #@param {type:"slider", min:0, max:3, step:0.5}
sampling_rate = 256 #@param {type:"number"}
n_channels = 20 #@param {type:"number"}
task = 'hmdd'
data_mode = 'cross_subject'

In [0]:
#@title ## DeepEEGAbstractor -Default params

model_name = 'Deep-EEG-Abstractor'

train_generator = FixedLenGenerator(batch_size=batch_size,
                                    duration=instance_duration,
                                    overlap=instance_overlap,
                                    sampling_rate=sampling_rate,
                                    is_train=True)

test_generator = FixedLenGenerator(batch_size=8,
                                   duration=instance_duration,
                                   overlap=instance_overlap,
                                   sampling_rate=sampling_rate,
                                   is_train=False)

params = {'task': task,
          'data_mode': data_mode,
          'main_res_dir': cv_results_dir,
          'model_name': model_name,
          'epochs': epochs,
          'train_generator': train_generator,
          'test_generator': test_generator,
          't': t,
          'k': k,
          'channel_drop': True}

validator = CrossValidator(**params)

dataloader = DataLoader(data_dir,
                        task,
                        data_mode,
                        sampling_rate,
                        instance_duration,
                        instance_overlap)
data, labels = dataloader.load_data()

input_shape = (sampling_rate * instance_duration,
               n_channels)

model_obj = DeepEEGAbstractor(input_shape,
                              model_name=model_name)

scores = validator.do_cv(model_obj,
                         data,
                         labels)

In [0]:
#@title ## DeepEEGAbstractor - BatchNormalization

model_name = 'Deep-EEG-Abstractor-BN'

train_generator = FixedLenGenerator(batch_size=batch_size,
                                    duration=instance_duration,
                                    overlap=instance_overlap,
                                    sampling_rate=sampling_rate,
                                    is_train=True)

test_generator = FixedLenGenerator(batch_size=8,
                                   duration=instance_duration,
                                   overlap=instance_overlap,
                                   sampling_rate=sampling_rate,
                                   is_train=False)

params = {'task': task,
          'data_mode': data_mode,
          'main_res_dir': cv_results_dir,
          'model_name': model_name,
          'epochs': epochs,
          'train_generator': train_generator,
          'test_generator': test_generator,
          't': t,
          'k': k,
          'channel_drop': True}

validator = CrossValidator(**params)

dataloader = DataLoader(data_dir,
                        task,
                        data_mode,
                        sampling_rate,
                        instance_duration,
                        instance_overlap)
data, labels = dataloader.load_data()

input_shape = (sampling_rate * instance_duration,
               n_channels)

model_obj = DeepEEGAbstractor(input_shape,
                              model_name=model_name,
                              normalization='batch')

scores = validator.do_cv(model_obj,
                         data,
                         labels)

In [0]:
#@title ## DeepEEGAbstractor - InstanceNormalization

model_name = 'Deep-EEG-Abstractor-IN'

train_generator = FixedLenGenerator(batch_size=batch_size,
                                    duration=instance_duration,
                                    overlap=instance_overlap,
                                    sampling_rate=sampling_rate,
                                    is_train=True)

test_generator = FixedLenGenerator(batch_size=8,
                                   duration=instance_duration,
                                   overlap=instance_overlap,
                                   sampling_rate=sampling_rate,
                                   is_train=False)

params = {'task': task,
          'data_mode': data_mode,
          'main_res_dir': cv_results_dir,
          'model_name': model_name,
          'epochs': epochs,
          'train_generator': train_generator,
          'test_generator': test_generator,
          't': t,
          'k': k,
          'channel_drop': True}

validator = CrossValidator(**params)

dataloader = DataLoader(data_dir,
                        task,
                        data_mode,
                        sampling_rate,
                        instance_duration,
                        instance_overlap)
data, labels = dataloader.load_data()

input_shape = (sampling_rate * instance_duration,
               n_channels)

model_obj = DeepEEGAbstractor(input_shape,
                              model_name=model_name,
                              normalization='instance')

scores = validator.do_cv(model_obj,
                         data,
                         labels)

In [0]:
#@title ## DeepEEGAbstractor - Deeper

model_name = 'Deep-EEG-Abstractor-Deeper'

train_generator = FixedLenGenerator(batch_size=batch_size,
                                    duration=instance_duration,
                                    overlap=instance_overlap,
                                    sampling_rate=sampling_rate,
                                    is_train=True)

test_generator = FixedLenGenerator(batch_size=8,
                                   duration=instance_duration,
                                   overlap=instance_overlap,
                                   sampling_rate=sampling_rate,
                                   is_train=False)

params = {'task': task,
          'data_mode': data_mode,
          'main_res_dir': cv_results_dir,
          'model_name': model_name,
          'epochs': epochs,
          'train_generator': train_generator,
          'test_generator': test_generator,
          't': t,
          'k': k,
          'channel_drop': True}

validator = CrossValidator(**params)

dataloader = DataLoader(data_dir,
                        task,
                        data_mode,
                        sampling_rate,
                        instance_duration,
                        instance_overlap)
data, labels = dataloader.load_data()

input_shape = (sampling_rate * instance_duration,
               n_channels)

model_obj = DeepEEGAbstractor(input_shape,
                              model_name=model_name,
                              n_kernels=(6, 6, 6, 4, 4))

scores = validator.do_cv(model_obj,
                         data,
                         labels)

In [0]:
#@title ## DeepEEGAbstractor - Wider

model_name = 'Deep-EEG-Abstractor-Wider'

train_generator = FixedLenGenerator(batch_size=batch_size,
                                    duration=instance_duration,
                                    overlap=instance_overlap,
                                    sampling_rate=sampling_rate,
                                    is_train=True)

test_generator = FixedLenGenerator(batch_size=8,
                                   duration=instance_duration,
                                   overlap=instance_overlap,
                                   sampling_rate=sampling_rate,
                                   is_train=False)

params = {'task': task,
          'data_mode': data_mode,
          'main_res_dir': cv_results_dir,
          'model_name': model_name,
          'epochs': epochs,
          'train_generator': train_generator,
          'test_generator': test_generator,
          't': t,
          'k': k,
          'channel_drop': True}

validator = CrossValidator(**params)

dataloader = DataLoader(data_dir,
                        task,
                        data_mode,
                        sampling_rate,
                        instance_duration,
                        instance_overlap)
data, labels = dataloader.load_data()

input_shape = (sampling_rate * instance_duration,
               n_channels)

model_obj = DeepEEGAbstractor(input_shape,
                              model_name=model_name,
                              n_kernels=(6, 6, 8, 10))

scores = validator.do_cv(model_obj,
                         data,
                         labels)

In [0]:
#@title ## DeepEEGAbstractor - Attv1

model_name = 'Deep-EEG-Abstractor-Attv1'

train_generator = FixedLenGenerator(batch_size=batch_size,
                                    duration=instance_duration,
                                    overlap=instance_overlap,
                                    sampling_rate=sampling_rate,
                                    is_train=True)

test_generator = FixedLenGenerator(batch_size=8,
                                   duration=instance_duration,
                                   overlap=instance_overlap,
                                   sampling_rate=sampling_rate,
                                   is_train=False)

params = {'task': task,
          'data_mode': data_mode,
          'main_res_dir': cv_results_dir,
          'model_name': model_name,
          'epochs': epochs,
          'train_generator': train_generator,
          'test_generator': test_generator,
          't': t,
          'k': k,
          'channel_drop': True}

validator = CrossValidator(**params)

dataloader = DataLoader(data_dir,
                        task,
                        data_mode,
                        sampling_rate,
                        instance_duration,
                        instance_overlap)
data, labels = dataloader.load_data()

input_shape = (sampling_rate * instance_duration,
               n_channels)

model_obj = DeepEEGAbstractor(input_shape,
                              model_name=model_name,
                              attention='v1')

scores = validator.do_cv(model_obj,
                         data,
                         labels)

In [0]:
#@title ## DeepEEGAbstractor - Attv2

model_name = 'Deep-EEG-Abstractor-Attv2'

train_generator = FixedLenGenerator(batch_size=batch_size,
                                    duration=instance_duration,
                                    overlap=instance_overlap,
                                    sampling_rate=sampling_rate,
                                    is_train=True)

test_generator = FixedLenGenerator(batch_size=8,
                                   duration=instance_duration,
                                   overlap=instance_overlap,
                                   sampling_rate=sampling_rate,
                                   is_train=False)

params = {'task': task,
          'data_mode': data_mode,
          'main_res_dir': cv_results_dir,
          'model_name': model_name,
          'epochs': epochs,
          'train_generator': train_generator,
          'test_generator': test_generator,
          't': t,
          'k': k,
          'channel_drop': True}

validator = CrossValidator(**params)

dataloader = DataLoader(data_dir,
                        task,
                        data_mode,
                        sampling_rate,
                        instance_duration,
                        instance_overlap)
data, labels = dataloader.load_data()

input_shape = (sampling_rate * instance_duration,
               n_channels)

model_obj = DeepEEGAbstractor(input_shape,
                              model_name=model_name,
                              attention='v2')

scores = validator.do_cv(model_obj,
                         data,
                         labels)

In [0]:
#@title ## DeepEEGAbstractor - Attv3

model_name = 'Deep-EEG-Abstractor-Attv3'

train_generator = FixedLenGenerator(batch_size=batch_size,
                                    duration=instance_duration,
                                    overlap=instance_overlap,
                                    sampling_rate=sampling_rate,
                                    is_train=True)

test_generator = FixedLenGenerator(batch_size=8,
                                   duration=instance_duration,
                                   overlap=instance_overlap,
                                   sampling_rate=sampling_rate,
                                   is_train=False)

params = {'task': task,
          'data_mode': data_mode,
          'main_res_dir': cv_results_dir,
          'model_name': model_name,
          'epochs': epochs,
          'train_generator': train_generator,
          'test_generator': test_generator,
          't': t,
          'k': k,
          'channel_drop': True}

validator = CrossValidator(**params)

dataloader = DataLoader(data_dir,
                        task,
                        data_mode,
                        sampling_rate,
                        instance_duration,
                        instance_overlap)
data, labels = dataloader.load_data()

input_shape = (sampling_rate * instance_duration,
               n_channels)

model_obj = DeepEEGAbstractor(input_shape,
                              model_name=model_name,
                              attention='v3')

scores = validator.do_cv(model_obj,
                         data,
                         labels)

In [0]:
#@title ## DeepEEGAbstractor - HDropout

model_name = 'Deep-EEG-Abstractor-HDropout'

train_generator = FixedLenGenerator(batch_size=batch_size,
                                    duration=instance_duration,
                                    overlap=instance_overlap,
                                    sampling_rate=sampling_rate,
                                    is_train=True)

test_generator = FixedLenGenerator(batch_size=8,
                                   duration=instance_duration,
                                   overlap=instance_overlap,
                                   sampling_rate=sampling_rate,
                                   is_train=False)

params = {'task': task,
          'data_mode': data_mode,
          'main_res_dir': cv_results_dir,
          'model_name': model_name,
          'epochs': epochs,
          'train_generator': train_generator,
          'test_generator': test_generator,
          't': t,
          'k': k,
          'channel_drop': True}

validator = CrossValidator(**params)

dataloader = DataLoader(data_dir,
                        task,
                        data_mode,
                        sampling_rate,
                        instance_duration,
                        instance_overlap)
data, labels = dataloader.load_data()

input_shape = (sampling_rate * instance_duration,
               n_channels)

model_obj = DeepEEGAbstractor(input_shape,
                              model_name=model_name,
                              spatial_dropout_rate=0.2,
                              dropout_rate=0.5)

scores = validator.do_cv(model_obj,
                         data,
                         labels)

In [0]:
#@title ## DeepEEGAbstractor - InputDropout

model_name = 'Deep-EEG-Abstractor-InputDropout'

train_generator = FixedLenGenerator(batch_size=batch_size,
                                    duration=instance_duration,
                                    overlap=instance_overlap,
                                    sampling_rate=sampling_rate,
                                    is_train=True)

test_generator = FixedLenGenerator(batch_size=8,
                                   duration=instance_duration,
                                   overlap=instance_overlap,
                                   sampling_rate=sampling_rate,
                                   is_train=False)

params = {'task': task,
          'data_mode': data_mode,
          'main_res_dir': cv_results_dir,
          'model_name': model_name,
          'epochs': epochs,
          'train_generator': train_generator,
          'test_generator': test_generator,
          't': t,
          'k': k,
          'channel_drop': True}

validator = CrossValidator(**params)

dataloader = DataLoader(data_dir,
                        task,
                        data_mode,
                        sampling_rate,
                        instance_duration,
                        instance_overlap)
data, labels = dataloader.load_data()

input_shape = (sampling_rate * instance_duration,
               n_channels)

model_obj = DeepEEGAbstractor(input_shape,
                              model_name=model_name,
                              input_dropout=True)

scores = validator.do_cv(model_obj,
                         data,
                         labels)